# Lab 6-1: Softmax Classification

## Intro
### problem
For each set of data, there is a list of `x` which could be classified into several catagories. Now calculate the possibility of each catagory.
### format
Answer `y` is given in the format of a single integer, representing the correct catagory.
### assumption
In this model, we first transform `x` into `x@W+b`, changing the shape of the 1D matrix into another 1D matrix whose size equals to the nuber of catagories. Then calcutate the possibility with softmax function to calculate the possibility. `W` and `b` are parameters to solve.

## Imports

In [77]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [78]:
# For reproducibility
torch.manual_seed(1)

## Softmax

Convert numbers to probabilities with softmax.

$$ P(class=i) = \frac{e^i}{\sum e^i} $$

In [79]:
z = torch.FloatTensor([1, 2, 3])

PyTorch has a `softmax` function.

In [80]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


Since they are probabilities, they should add up to 1. Let's do a sanity check.

In [81]:
hypothesis.sum()

tensor(1.)

## Cross Entropy Loss (Low-level)

For multi-class classification, we use the cross entropy loss.

$$ L = \frac{1}{N} \sum - y \log(\hat{y}) $$

where $\hat{y}$ is the predicted probability and $y$ is the correct probability (0 or 1).

In [82]:
z = torch.rand(3, 5, requires_grad=True)
print(z)
hypothesis = F.softmax(z, dim=1)
print(hypothesis, hypothesis.sum(dim=1))

tensor([[0.7576, 0.2793, 0.4031, 0.7347, 0.0293],
        [0.7999, 0.3971, 0.7544, 0.5695, 0.4388],
        [0.6387, 0.5247, 0.6826, 0.3051, 0.4635]], requires_grad=True)
tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>) tensor([1.0000, 1.0000, 1.0000], grad_fn=<SumBackward1>)


*This is what will happen when dim set to 0.*

In [83]:
hypothesis1 = F.softmax(z, dim=0)
print(hypothesis1, hypothesis1.sum(dim=0))

tensor([[0.3412, 0.2938, 0.2671, 0.4002, 0.2469],
        [0.3559, 0.3306, 0.3796, 0.3393, 0.3719],
        [0.3029, 0.3756, 0.3533, 0.2605, 0.3812]], grad_fn=<SoftmaxBackward0>) tensor([1., 1., 1., 1., 1.], grad_fn=<SumBackward1>)


In [84]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 2, 1])


**torch.randint**

``` python
    torch.randint(low, high, size, *, generator=None, out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False)
```
- low (int): The lower bound (inclusive) of the range of random integers.
- high (int): The upper bound (exclusive) of the range of random integers.
- size (tuple of ints): The shape of the output tensor.
- generator (torch.Generator, optional): A pseudorandom number generator for sampling.
- out (Tensor, optional): The output tensor.
- dtype (torch.dtype, optional): The desired data type of the returned tensor. Default is torch.int64.
- layout (torch.layout, optional): The desired layout of the returned tensor. Default is torch.strided.
- device (torch.device, optional): The desired device of the returned tensor. Default is the current device for the default tensor type.
- requires_grad (bool, optional): If autograd should record operations on the returned tensor. Default is False.

In [85]:
y_one_hot = torch.zeros_like(hypothesis) # init with zeros
y_one_hot.scatter_(1, y.unsqueeze(1), 1) # dim=1, index=y.unsqueeze(1), src=1

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [86]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


## Cross-entropy Loss with `torch.nn.functional`

PyTorch has `F.log_softmax()` function.

In [87]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)

In [88]:
# High level
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

PyTorch also has `F.nll_loss()` function that computes the negative loss likelihood.

In [89]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [90]:
# High level
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

PyTorch also has `F.cross_entropy` that combines `F.log_softmax()` and `F.nll_loss()`.

In [91]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

## Training with Low-level Cross Entropy Loss

In [92]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [93]:
# initialize W, b
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer initialize
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost calc (1)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

    # optimize
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # log
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


## Training with `F.cross_entropy`

In [ ]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost calc (2)
    z = x_train.matmul(W) + b # or .mm or @
    cost = F.cross_entropy(z, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568256
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


## High-level Implementation with `nn.Module`

In [95]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) #input size: 4, output size: 3

    def forward(self, x):
        return self.linear(x)

In [96]:
model = SoftmaxClassifierModel()

Let's try another new dataset.

In [103]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    prediction = model(x_train)

    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))        

Epoch    0/1000 Cost: 0.112533
Epoch  100/1000 Cost: 0.109683
Epoch  200/1000 Cost: 0.106970
Epoch  300/1000 Cost: 0.104385
Epoch  400/1000 Cost: 0.101918
Epoch  500/1000 Cost: 0.099563
Epoch  600/1000 Cost: 0.097311
Epoch  700/1000 Cost: 0.095157
Epoch  800/1000 Cost: 0.093093
Epoch  900/1000 Cost: 0.091116
Epoch 1000/1000 Cost: 0.089218
